In [1]:
import pandas as pd
import numpy as np
from collections import *

In [2]:
_map = pd.read_csv('cik-cusip_1993-2017.csv',names=['date','cik','cusip','file'],usecols=['date','cik','cusip'])

In [3]:
_map['date'] = pd.to_datetime(_map.date,format='%Y-%m-%d')

In [4]:
_map['fyear'] = _map.date.apply(lambda x: x.year - 1  if x.month<7 else x.year)

In [5]:
map_ = _map.groupby(['cik','fyear']).cusip.agg(lambda x:Counter(x.values).most_common()[0][0]).reset_index()

In [6]:
rebuilt = []
for x in map_.cik.unique():
    for yr in range(1993,2020):
        rebuilt.append([x,yr])
rebuilt = pd.DataFrame(rebuilt)
rebuilt.columns = ['cik','fyear']

In [7]:
map_ = rebuilt.merge(map_,on=['cik','fyear'],how='left')

In [8]:
map_['cusip'] = map_.groupby(['cik']).cusip.fillna(method='ffill').fillna(method='bfill')

In [9]:
map_['cusip'] = map_['cusip'].map(lambda x:x[:8] if len(x)==9 else x)

In [10]:
map_.to_csv('cik-cusip_mapping_1993-2018.csv')